In [35]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier

In [36]:
data = pd.read_csv("data_behavior_expand_x30.csv")

In [37]:
data.head(3)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),State,Size,Color,Season,...,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,id,City,Region
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,1,Louisville,South
1,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,2,Lexington,South
2,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,3,Louisville,South


In [38]:
df = data.drop(labels=["Shipping Type","Promo Code Used"], axis = 1)
df.head(3)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),State,Size,Color,Season,Review Rating,Subscription Status,Discount Applied,Previous Purchases,Payment Method,Frequency of Purchases,id,City,Region
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Yes,14,Venmo,Fortnightly,1,Louisville,South
1,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Yes,14,Venmo,Fortnightly,2,Lexington,South
2,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Yes,14,Venmo,Fortnightly,3,Louisville,South


In [39]:
print(df['Category'].unique())
print("*"*70)
print(df['State'].unique())
print("*"*70)
print(df['Color'].unique())
print("*"*70)
print(df['Season'].unique())
print("*"*70)
print(df['Gender'].unique())
print("*"*70)
print(df['Subscription Status'].unique())
print("*"*70)
print(df['Frequency of Purchases'].unique())
print("*"*70)
print(df['Item Purchased'].unique())


['Clothing' 'Footwear' 'Outerwear' 'Accessories']
**********************************************************************
['Kentucky' 'Maine' 'Massachusetts' 'Rhode Island' 'Oregon' 'Wyoming'
 'Montana' 'Louisiana' 'West Virginia' 'Missouri' 'Arkansas' 'Hawaii'
 'Delaware' 'New Hampshire' 'New York' 'Alabama' 'Mississippi'
 'North Carolina' 'California' 'Oklahoma' 'Florida' 'Texas' 'Nevada'
 'Kansas' 'Colorado' 'North Dakota' 'Illinois' 'Indiana' 'Arizona'
 'Alaska' 'Tennessee' 'Ohio' 'New Jersey' 'Maryland' 'Vermont'
 'New Mexico' 'South Carolina' 'Idaho' 'Pennsylvania' 'Connecticut' 'Utah'
 'Virginia' 'Georgia' 'Nebraska' 'Iowa' 'South Dakota' 'Minnesota'
 'Washington' 'Wisconsin' 'Michigan']
**********************************************************************
['Gray' 'Maroon' 'Turquoise' 'White' 'Charcoal' 'Silver' 'Pink' 'Purple'
 'Olive' 'Gold' 'Violet' 'Teal' 'Lavender' 'Black' 'Green' 'Peach' 'Red'
 'Cyan' 'Brown' 'Beige' 'Orange' 'Indigo' 'Yellow' 'Magenta' 'Blue']
***********

In [40]:
names = df.columns
types = df.dtypes

scaler = MinMaxScaler()

label_encoders = {}
for col in names:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


# Dự Đoán Xu Hướng Thời Trang

In [41]:
X_trend = df[['Age', 'Gender', 'Category', 'State',  'Season']]
Y_trend = df['Item Purchased']
X_trend_scaled = scaler.fit_transform(X_trend)
model_trend = RandomForestClassifier(random_state=42)
model_trend.fit(X_trend_scaled, Y_trend)


RandomForestClassifier(random_state=42)

In [42]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Giả sử scaler và label_encoders đã được khởi tạo và huấn luyện trước đó

def predict_trend_item(age, gender, category, state, season):
    new_customer_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Category': [category],
        'State': [state],
        'Season': [season]
    })

    # Mã hóa dữ liệu mới
    for col in ['Age', 'Gender', 'Category', 'State', 'Season']:
        # Kiểm tra nếu giá trị có trong label encoder
        if new_customer_data[col][0] not in label_encoders[col].classes_:
            print(f"Giá trị '{new_customer_data[col][0]}' không có trong label encoder của cột '{col}'")
            return None
        new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])

    # Đảm bảo rằng new_customer_data chỉ chứa các cột cần thiết
    new_customer_data = new_customer_data[['Age', 'Gender', 'Category', 'State', 'Season']]

    # Chuẩn hóa dữ liệu mới
    new_customer_data_scaled = scaler.transform(new_customer_data)

    # Dự đoán sản phẩm
    predicted_item = model_trend.predict(new_customer_data_scaled)

    # Chuyển đổi dự đoán về tên sản phẩm gốc
    predicted_item_name = label_encoders['Item Purchased'].inverse_transform(predicted_item)

    # Hiển thị kết quả dự đoán
    print("Dự đoán sản phẩm cho khách hàng:", predicted_item_name)

    # Tính toán tỷ lệ phần trăm
    percentage = calculate_preference_percentage(predicted_item_name[0], age, season, category)
    print(f"Tỷ lệ phần trăm khách hàng có sở thích giống nhau: {percentage:.2f}%")

    return predicted_item_name

def calculate_preference_percentage(item_name, age, season, category):
    # Lọc dữ liệu theo độ tuổi
    total = data[data['Age'] == age]
    total = total[total['Season'] == season]
    total = total[total['Category'] == category]
    filtered_customers = total[total['Item Purchased'] == item_name]
    if filtered_customers.empty:
        print(f"Không có khách hàng nào trong độ tuổi {age}.")
        return 0.0
    

    percentage = (filtered_customers.shape[0]/total.shape[0]) * 100
    return percentage

In [43]:
item = predict_trend_item(30, 'Female', 'Accessories', 'Kentucky','Summer')

Dự đoán sản phẩm cho khách hàng: ['Belt']
Tỷ lệ phần trăm khách hàng có sở thích giống nhau: 50.00%


# Dự đoán phân khúc khách hàng

In [44]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [45]:
# Chọn các cột để phân cụm
X_class = df[['Age', 'Gender', 'Purchase Amount (USD)', 'State', 'Subscription Status', 'Frequency of Purchases']]

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_scaled_class = scaler.fit_transform(X_class)

# Thực hiện K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=42)  # Chia thành 2 nhóm: tiềm năng và không tiềm năng
df['Segment'] = kmeans.fit_predict(X_scaled_class)

# Tính khoảng cách Euclidean từ mỗi khách hàng đến các trung tâm cụm
centroids = kmeans.cluster_centers_
distances = np.linalg.norm(X_scaled_class - centroids[kmeans.labels_], axis=1)

# Thêm khoảng cách vào DataFrame
df['Distance to Centroid'] = distances

# Xác định ngưỡng (threshold) cho khoảng cách
threshold = np.median(distances)  # hoặc bạn có thể chọn giá trị khác tùy ý

# Phân loại khách hàng
df['Potential Customer'] = np.where(df['Distance to Centroid'] < threshold, 'Tiềm năng', 'Không tiềm năng')
data['Potential Customer'] = df['Potential Customer']

In [46]:
# Giả sử df đã được định nghĩa và chứa dữ liệu cần thiết
X_potential = df[['Age', 'Gender', 'Purchase Amount (USD)', 'State', 'Subscription Status', 'Frequency of Purchases']]
Y_potential = df['Potential Customer']

# Mã hóa Y_potential
label_encoders['Potential Customer'] = LabelEncoder()
Y_potential_encoded = label_encoders['Potential Customer'].fit_transform(Y_potential)

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_potential_scaled = scaler.fit_transform(X_potential)

# Huấn luyện mô hình
model_potential = RandomForestClassifier(random_state=42)
model_potential.fit(X_potential_scaled, Y_potential_encoded)


RandomForestClassifier(random_state=42)

In [47]:
import pandas as pd

def predict_potential_customer(age, gender, purchase_amount, state, subscription_status, frequency_of_purchases):
    # Tạo DataFrame cho khách hàng mới
    new_customer_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Purchase Amount (USD)': [purchase_amount],
        'State': [state],
        'Subscription Status': [subscription_status],
        'Frequency of Purchases': [frequency_of_purchases]
    })

    # Mã hóa dữ liệu mới cho các cột cần thiết
    for col in ['Age', 'Gender', 'State', 'Subscription Status', 'Frequency of Purchases']:
        if col in label_encoders:
            new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])
        else:
            print(f"Không tìm thấy label encoder cho cột: {col}")

    # Đảm bảo rằng new_customer_data chỉ chứa các cột cần thiết
    new_customer_data = new_customer_data[['Age', 'Gender', 'Purchase Amount (USD)', 'State', 'Subscription Status', 'Frequency of Purchases']]

    # Chuẩn hóa dữ liệu mới
    new_customer_data_scaled = scaler.transform(new_customer_data)

    # Dự đoán tiềm năng
    predicted_potential = model_potential.predict(new_customer_data_scaled)

    # Chuyển đổi dự đoán về tên nhóm tiềm năng
    predicted_potential_name = label_encoders['Potential Customer'].inverse_transform(predicted_potential)

    # Tính toán tỷ lệ phần trăm
    print("Dự đoán tiềm năng khách hàng:", predicted_potential_name[0])
    percentage = percentage_potential(age, gender, frequency_of_purchases, predicted_potential_name[0])
    
    # Hiển thị kết quả dự đoán
   
def percentage_potential(age, gender, frequency, potential_name):
    # Lọc dữ liệu theo tên nhóm tiềm năng
    total = data[data['Potential Customer'] == potential_name]
    
    # Kiểm tra nếu không có khách hàng nào trong nhóm đó
    if total.empty:
        return 0.0
    
    # Lọc theo độ tuổi
    total_age = total[total['Age'] == age]
    
    # Lọc theo giới tính
    total_gender = total[total['Gender'] == gender]

    # Lọc theo tần suất mua hàng
    total_frequency = total[total['Frequency of Purchases'] == frequency]

    # Tính toán tỷ lệ phần trăm
    percentage_age = (total_age.shape[0] / total.shape[0]) * 100
    percentage_gender = (total_gender.shape[0] / total.shape[0]) * 100
    percentage_frequency = (total_frequency.shape[0] / total.shape[0]) * 100
    print(f"Tỷ lệ phần trăm khách hàng ở tuổi '{age}' thuộc nhóm '{potential_name}': {percentage_age:.2f}%")
    print(f"Tỷ lệ phần trăm khách hàng ở tuổi '{gender}' thuộc nhóm '{potential_name}': {percentage_gender:.2f}%")
    print(f"Tỷ lệ phần trăm khách hàng ở tuổi '{frequency}' thuộc nhóm '{potential_name}': {percentage_frequency:.2f}%")
    return percentage_age

In [48]:
predict_potential_customer(44, 'Female', 77, 'Minnesota', "No",'Weekly')


Dự đoán tiềm năng khách hàng: Tiềm năng
Tỷ lệ phần trăm khách hàng ở tuổi '44' thuộc nhóm 'Tiềm năng': 1.95%
Tỷ lệ phần trăm khách hàng ở tuổi 'Female' thuộc nhóm 'Tiềm năng': 52.26%
Tỷ lệ phần trăm khách hàng ở tuổi 'Weekly' thuộc nhóm 'Tiềm năng': 6.62%


In [49]:
import joblib

# Chuẩn hóa dữ liệu cho xu hướng thời trang
scaler_trend = MinMaxScaler()
X_trend_scaled = scaler_trend.fit_transform(X_trend)

# Chuẩn hóa dữ liệu cho phân khúc khách hàng
scaler_potential = MinMaxScaler()
X_potential_scaled = scaler_potential.fit_transform(X_potential)

# Lưu scaler riêng cho từng mô hình
joblib.dump(scaler_trend, "models/scaler_trend.pkl")
joblib.dump(scaler_potential, "models/scaler_potential.pkl")


['models/scaler_potential.pkl']

In [50]:
import joblib

# Lưu model dự đoán xu hướng
joblib.dump(model_trend, "models/model_trend.pkl")

# Lưu model dự đoán phân khúc
joblib.dump(model_potential, "models/model_potential.pkl")

# Lưu scaler riêng cho từng mô hình
joblib.dump(scaler_trend, "models/scaler_trend.pkl")
joblib.dump(scaler_potential, "models/scaler_potential.pkl")

# Lưu label encoders
joblib.dump(label_encoders, "models/label_encoders.pkl")


['models/label_encoders.pkl']

In [51]:
scaler_trend = joblib.load("models/scaler_trend.pkl")
scaler_potential = joblib.load("models/scaler_potential.pkl")